In [96]:
import pandas as pd
import math
from iapws import IAPWS97
import sys
%run equations_module.ipynb
%run class_module.ipynb
sys.stdout = open("pipe network.log", "w")

In [97]:
#import data frame for pipe network;
net_df = pd.read_excel('bom.xls', sheetname=9, parse_cols='A:D')
#import data frame for node attributes
node_df = pd.read_excel('bom.xls', sheetname=8, parse_cols='A:D')
#import dataframe for pipeline attributes
pipe_df = pd.read_excel('bom.xls', sheetname=7)

In [98]:
network_graph = {}
node_dict = {}
pipe_dict = {}
#Create a graph from nodes; in dictionary format
for nodes in range (len(net_df.index)):
    nodeID, inlet, outlet, pflow = map(lambda x: net_df.iloc[nodes, x], range(0,4))
    network_graph[nodeID] = [inlet, outlet, pflow]

for nodes in range(len(node_df.index)):
    #create a node class for each row and store it in a dictionary
    nodeID, P, mf, z = map(lambda x: node_df.iloc[nodes, x], range(0,4))
    node_dict[nodeID] = node(nodeID, P, mf, z)

for pipelines in range(len(pipe_df.index)):
    #create a pipeline class for each row and store it in a dictionary
    lineID, length, in_dia, roughness, pipe_t, insul_t, pipe_k, insul_k, n_elbow45, n_elbow90, n_tee, \
        n_rbend, n_coupling, n_union, n_gatefull, n_gatehalf, out_dia = \
        map(lambda x: pipe_df.iloc[pipelines, x], range(0,17))   
    pipe_dict[lineID] = pipeline(lineID, length, in_dia, roughness, pipe_t, insul_t, pipe_k, insul_k, \
        n_elbow45, n_elbow90, n_tee, n_rbend, n_coupling, n_union, n_gatefull, n_gatehalf, out_dia)

In [99]:
node_out={}
for x in sorted(network_graph):
    
    #edge[x][0] for input node; edge[x][1] for output node; x for pipeline
    print('SteamLine', x, network_graph[x])
    print('Percent Flow: ', network_graph[x][2])

    t_loss = pressure_drop(node_dict[network_graph[x][0]].z, node_dict[network_graph[x][1]].z, pipe_dict[x].in_dia, \
                       node_dict[network_graph[x][0]].mf, \
                       node_dict[network_graph[x][0]].P, pipe_dict[x].length, pipe_dict[x].n_elbow45, \
                       pipe_dict[x].n_elbow90, pipe_dict[x].n_gatefull, pipe_dict[x].insul_k, \
                       pipe_dict[x].insul_t, pipe_dict[x].out_dia, pipe_dict[x].roughness)

    if node_dict[network_graph[x][1]].P == 0:
        node_dict[network_graph[x][1]].P = t_loss[0]
    else:
        node_dict[network_graph[x][1]].P = (node_dict[network_graph[x][1]].P + t_loss[0] )/2
    node_dict[network_graph[x][1]].mf += t_loss[1] * network_graph[x][2]
    print('Steam Flow Out: ', node_dict[network_graph[x][1]].mf)
    #print(pipeline_dict[x].length, node_dict[edge[x][1]].mf)
    #print(x)
    node_out[x]= [node_dict[network_graph[x][1]].P, node_dict[network_graph[x][1]].mf]
    #add other input and output information in the dictionary
    for n in t_loss[2]:
        node_out[x].append(n)   
    

In [100]:
'''Export results to excel file'''
df_out = pd.DataFrame(node_out)
df_out = df_out.transpose()

df_out.columns=['Pressure', 'SteamFlow', 'Velocity', 'TotalLoss', 'HeatLoss', 'SkinFriction', 'Expansion', \
                'Contraction', 'Fittings', 'InletPressure', 'OutletPressure', 'SF_in', 'SF_out', 'K.E.', 'P.E.', \
                'Length', '#Fittings']

writer = pd.ExcelWriter('model_results.xls')
df_out.to_excel(writer,'results')
writer.save()